# 📊 Probability Foundations

This notebook builds intuition for:
1. Probability vs probability distribution
2. Random variables (discrete vs continuous)
3. PMF vs PDF - why the difference?
4. The discrete → continuous transition
5. Sampling and visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.integrate import quad

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 12

np.random.seed(42)

## 1. Probability vs Probability Distribution

**Probability** = a single number answering one question

**Distribution** = the complete picture (all probabilities for all outcomes)

In [ ]:
# Example: Fair die
# The DISTRIBUTION is the complete specification
die_distribution = {1: 1/6, 2: 1/6, 3: 1/6, 4: 1/6, 5: 1/6, 6: 1/6}

print("The DISTRIBUTION (complete picture):")
print("="*40)
for outcome, prob in die_distribution.items():
    print(f"  P(X = {outcome}) = {prob:.4f}")

print(f"\nSum of all probabilities: {sum(die_distribution.values()):.4f}")

print("\n" + "="*40)
print("Individual PROBABILITIES (single questions):")
print("="*40)
print(f"  P(roll a 3) = {die_distribution[3]:.4f}")
print(f"  P(roll even) = {die_distribution[2] + die_distribution[4] + die_distribution[6]:.4f}")
print(f"  P(roll ≤ 2) = {die_distribution[1] + die_distribution[2]:.4f}")

print("\n🔑 Key insight: Distribution answers ALL questions. Probability answers ONE.")

In [ ]:
# Visualize the difference
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: The full distribution
ax = axes[0]
outcomes = list(die_distribution.keys())
probs = list(die_distribution.values())
ax.bar(outcomes, probs, color='steelblue', edgecolor='black', alpha=0.7)
ax.set_xlabel('Outcome')
ax.set_ylabel('Probability')
ax.set_title('DISTRIBUTION: Complete Picture\n(Answers any question you might ask)')
ax.set_ylim(0, 0.3)
ax.set_xticks(outcomes)

# Right: One specific probability
ax = axes[1]
colors = ['lightgray'] * 6
colors[2] = 'coral'  # Highlight "roll a 3"
ax.bar(outcomes, probs, color=colors, edgecolor='black', alpha=0.7)
ax.set_xlabel('Outcome')
ax.set_ylabel('Probability')
ax.set_title('PROBABILITY: Single Question\nP(X = 3) = 1/6 ≈ 0.167')
ax.set_ylim(0, 0.3)
ax.set_xticks(outcomes)
ax.annotate('This one!', xy=(3, 1/6), xytext=(4, 0.22),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=12, color='red')

plt.tight_layout()
plt.show()

## 2. Discrete vs Continuous: The Fundamental Difference

**Discrete:** Countable outcomes (can list them: 1, 2, 3, ...)

**Continuous:** Uncountable outcomes (any real number in an interval)

In [ ]:
# The continuous "paradox"
print("THE CONTINUOUS PARADOX")
print("="*50)
print("\nQuestion: If X is height (continuous), what is P(X = 170.5)?")
print("\nLet's think about it...")
print("\n1. X could be 170.5000000000000...")
print("2. X could be 170.5000000000001...")
print("3. X could be 170.4999999999999...")
print("4. There are INFINITELY many possible values!")
print("\n5. All probabilities must sum to 1.")
print("6. If each exact value had probability ε > 0,")
print("   the sum would be ∞ × ε = ∞ ≠ 1 ❌")
print("\n🔑 CONCLUSION: P(X = exact value) = 0 for continuous X!")
print("\nBut wait... then how do we describe continuous distributions?")
print("→ We use DENSITY instead of mass!")

In [ ]:
# Visualize: Discrete PMF vs Continuous PDF
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Discrete PMF (like point masses)
ax = axes[0]
x_discrete = np.arange(1, 7)
pmf = np.ones(6) / 6

# Draw as "lollipops" (point masses)
ax.vlines(x_discrete, 0, pmf, colors='steelblue', linewidth=3)
ax.scatter(x_discrete, pmf, s=100, c='steelblue', zorder=5)

ax.set_xlabel('x')
ax.set_ylabel('P(X = x)  [probability]')
ax.set_title('DISCRETE: PMF (Probability Mass Function)\nEach point has actual probability')
ax.set_ylim(0, 0.3)
ax.set_xlim(0, 7)

# Annotation
ax.annotate('P(X=3) = 1/6\n(actual probability)', xy=(3, 1/6), xytext=(4.5, 0.22),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=11, color='red')

# Right: Continuous PDF (like a fluid)
ax = axes[1]
x_continuous = np.linspace(-4, 4, 1000)
pdf = stats.norm.pdf(x_continuous, 0, 1)

ax.fill_between(x_continuous, pdf, alpha=0.3, color='steelblue')
ax.plot(x_continuous, pdf, 'b-', linewidth=2)

ax.set_xlabel('x')
ax.set_ylabel('f(x)  [density, NOT probability!]')
ax.set_title('CONTINUOUS: PDF (Probability Density Function)\nNo single point has probability')
ax.set_ylim(0, 0.5)

# Highlight that a single point has zero probability
ax.axvline(x=0.5, color='red', linestyle='--', alpha=0.7)
ax.annotate('P(X = 0.5) = 0\n(exactly zero!)', xy=(0.5, 0.35), xytext=(1.5, 0.4),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=11, color='red')

plt.tight_layout()
plt.show()

print("\n🔑 PMF gives probability. PDF gives DENSITY (probability per unit length).")

## 3. PDF: Probability = Area Under Curve

For continuous distributions:
$$P(a \leq X \leq b) = \int_a^b f(x) \, dx = \text{Area under curve from } a \text{ to } b$$

In [ ]:
# Demonstrate: Probability = Area
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

x = np.linspace(-4, 4, 1000)
pdf = stats.norm.pdf(x, 0, 1)

# Example 1: P(-1 < X < 1)
ax = axes[0]
ax.plot(x, pdf, 'b-', linewidth=2)
mask = (x >= -1) & (x <= 1)
ax.fill_between(x[mask], pdf[mask], alpha=0.5, color='coral')
prob = stats.norm.cdf(1) - stats.norm.cdf(-1)
ax.set_title(f'P(-1 ≤ X ≤ 1) = {prob:.4f}\nArea of shaded region')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')

# Example 2: P(X < 0)
ax = axes[1]
ax.plot(x, pdf, 'b-', linewidth=2)
mask = x <= 0
ax.fill_between(x[mask], pdf[mask], alpha=0.5, color='coral')
prob = stats.norm.cdf(0)
ax.set_title(f'P(X ≤ 0) = {prob:.4f}\nHalf the area (symmetric)')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')

# Example 3: P(X > 1.96)
ax = axes[2]
ax.plot(x, pdf, 'b-', linewidth=2)
mask = x >= 1.96
ax.fill_between(x[mask], pdf[mask], alpha=0.5, color='coral')
prob = 1 - stats.norm.cdf(1.96)
ax.set_title(f'P(X ≥ 1.96) = {prob:.4f}\n"Tail probability"')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')

plt.tight_layout()
plt.show()

print("Key insight: For continuous distributions, we can only ask about INTERVALS.")
print("The probability equals the integral (area) over that interval.")

In [ ]:
# Show that PDF can exceed 1 (it's density, not probability!)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Narrow uniform distribution
ax = axes[0]
a, b = 0, 0.4  # Width = 0.4
x = np.linspace(-0.5, 1, 1000)
pdf = np.where((x >= a) & (x <= b), 1/(b-a), 0)

ax.fill_between(x, pdf, alpha=0.5, color='steelblue')
ax.plot(x, pdf, 'b-', linewidth=2)
ax.axhline(y=1, color='red', linestyle='--', label='y = 1')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title(f'Uniform(0, 0.4): f(x) = {1/(b-a):.1f}\nDensity > 1, but area = 1')
ax.set_ylim(0, 3)
ax.legend()

# Very narrow Gaussian
ax = axes[1]
x = np.linspace(-1, 1, 1000)
sigma = 0.2
pdf = stats.norm.pdf(x, 0, sigma)

ax.fill_between(x, pdf, alpha=0.5, color='steelblue')
ax.plot(x, pdf, 'b-', linewidth=2)
ax.axhline(y=1, color='red', linestyle='--', label='y = 1')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title(f'N(0, {sigma}²): max f(x) = {1/(sigma*np.sqrt(2*np.pi)):.2f}\nDensity > 1, but integral = 1')
ax.legend()

plt.tight_layout()
plt.show()

print("\n🔑 PDF values > 1 are perfectly valid!")
print("What matters: ∫f(x)dx = 1 (total area = 1)")
print("Narrow distributions = tall but thin = still area 1")

## 4. The Discrete → Continuous Transition

Understanding this transition is KEY for deriving continuous formulas from discrete ones.

In [ ]:
def demonstrate_discrete_to_continuous(n_bins_list=[5, 10, 20, 50]):
    """Show how discrete histogram approaches continuous PDF."""
    
    # True continuous distribution: N(0, 1)
    x_cont = np.linspace(-4, 4, 1000)
    pdf_true = stats.norm.pdf(x_cont, 0, 1)
    
    fig, axes = plt.subplots(1, len(n_bins_list), figsize=(16, 4))
    
    for ax, n_bins in zip(axes, n_bins_list):
        # Create discrete approximation
        bin_edges = np.linspace(-4, 4, n_bins + 1)
        bin_width = bin_edges[1] - bin_edges[0]
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        
        # PMF of discretized distribution (probability in each bin)
        pmf = stats.norm.cdf(bin_edges[1:]) - stats.norm.cdf(bin_edges[:-1])
        
        # Plot as bar chart (histogram style)
        ax.bar(bin_centers, pmf/bin_width, width=bin_width*0.9, 
               alpha=0.5, color='steelblue', edgecolor='black')
        
        # Overlay true PDF
        ax.plot(x_cont, pdf_true, 'r-', linewidth=2, label='True PDF')
        
        ax.set_xlabel('x')
        ax.set_ylabel('Density')
        ax.set_title(f'{n_bins} bins (Δx = {bin_width:.2f})')
        ax.set_xlim(-4, 4)
        ax.set_ylim(0, 0.5)
    
    axes[-1].legend()
    plt.suptitle('Discrete → Continuous: As bins get finer, histogram → PDF', fontsize=14, y=1.05)
    plt.tight_layout()
    plt.show()

demonstrate_discrete_to_continuous()

print("\n🔑 Key relationship:")
print("   PMF ≈ PDF × Δx")
print("   p(xᵢ) ≈ f(xᵢ) · Δx")
print("\nAs Δx → 0:")
print("   Σ → ∫")
print("   p(x) → f(x)dx")

In [ ]:
# Derive the mean formula: discrete → continuous
print("DERIVING E[X]: Discrete to Continuous")
print("="*50)
print("\n1. DISCRETE formula:")
print("   E[X] = Σᵢ xᵢ · p(xᵢ)")
print("\n2. Approximate continuous with bins of width Δx:")
print("   For each bin centered at xᵢ:")
print("   p(xᵢ) ≈ f(xᵢ) · Δx")
print("\n3. Substitute:")
print("   E[X] ≈ Σᵢ xᵢ · f(xᵢ) · Δx")
print("\n4. Take limit as Δx → 0:")
print("   E[X] = ∫ x · f(x) dx")
print("\n" + "="*50)
print("🔑 This pattern works for EVERYTHING:")
print("   Variance, Entropy, KL divergence, etc.")
print("   Just replace Σ with ∫ and p(x) with f(x)dx")

In [ ]:
# Numerical demonstration: Mean converges
def compute_mean_discrete_approximation(n_bins):
    """Approximate E[X] using discretization."""
    bin_edges = np.linspace(-6, 6, n_bins + 1)
    bin_width = bin_edges[1] - bin_edges[0]
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # PMF of discretized N(0,1)
    pmf = stats.norm.cdf(bin_edges[1:]) - stats.norm.cdf(bin_edges[:-1])
    
    # E[X] = Σ x · p(x)
    mean_approx = np.sum(bin_centers * pmf)
    return mean_approx, bin_width

# True mean of N(0,1) is 0
true_mean = 0

print("Computing E[X] for N(0,1) using discretization:")
print("="*50)
print(f"True mean: {true_mean}")
print(f"\n{'n_bins':<10} {'Δx':<10} {'E[X] approx':<15} {'Error':<10}")
print("-"*50)

for n_bins in [5, 10, 20, 50, 100, 500, 1000]:
    mean_approx, dx = compute_mean_discrete_approximation(n_bins)
    error = abs(mean_approx - true_mean)
    print(f"{n_bins:<10} {dx:<10.4f} {mean_approx:<15.6f} {error:<10.2e}")

print("\n→ As bins increase (Δx decreases), approximation → exact!")

## 5. CDF: The Universal Bridge

The CDF works the same way for both discrete and continuous!

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Top row: Discrete
# PMF
ax = axes[0, 0]
x_discrete = np.arange(1, 7)
pmf = np.ones(6) / 6
ax.vlines(x_discrete, 0, pmf, colors='steelblue', linewidth=3)
ax.scatter(x_discrete, pmf, s=100, c='steelblue', zorder=5)
ax.set_xlabel('x')
ax.set_ylabel('P(X = x)')
ax.set_title('Discrete PMF: Fair Die')
ax.set_xlim(0, 7)
ax.set_ylim(0, 0.3)

# CDF
ax = axes[0, 1]
x_cdf = np.arange(0, 8, 0.01)
cdf = np.zeros_like(x_cdf)
for i, val in enumerate([1, 2, 3, 4, 5, 6]):
    cdf[x_cdf >= val] = (i + 1) / 6

ax.plot(x_cdf, cdf, 'b-', linewidth=2)
# Mark the jumps
for i in range(1, 7):
    ax.plot(i, i/6, 'bo', markersize=8)
    if i > 1:
        ax.plot(i, (i-1)/6, 'bo', markersize=8, fillstyle='none')

ax.set_xlabel('x')
ax.set_ylabel('F(x) = P(X ≤ x)')
ax.set_title('Discrete CDF: Staircase function')
ax.set_xlim(0, 7)
ax.set_ylim(0, 1.1)
ax.grid(True, alpha=0.3)

# Bottom row: Continuous
# PDF
ax = axes[1, 0]
x_cont = np.linspace(-4, 4, 1000)
pdf = stats.norm.pdf(x_cont, 0, 1)
ax.fill_between(x_cont, pdf, alpha=0.3, color='steelblue')
ax.plot(x_cont, pdf, 'b-', linewidth=2)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title('Continuous PDF: N(0, 1)')

# CDF
ax = axes[1, 1]
cdf_cont = stats.norm.cdf(x_cont, 0, 1)
ax.plot(x_cont, cdf_cont, 'b-', linewidth=2)
ax.set_xlabel('x')
ax.set_ylabel('F(x) = P(X ≤ x)')
ax.set_title('Continuous CDF: Smooth S-curve')
ax.set_ylim(0, 1.1)
ax.grid(True, alpha=0.3)

# Add horizontal lines at key points
for p in [0.5, 0.84, 0.975]:
    x_val = stats.norm.ppf(p)
    ax.axhline(y=p, xmax=(x_val+4)/8, color='red', linestyle=':', alpha=0.5)
    ax.axvline(x=x_val, ymax=p, color='red', linestyle=':', alpha=0.5)

plt.tight_layout()
plt.show()

print("\n🔑 CDF properties (same for discrete AND continuous):")
print("   • F(-∞) = 0")
print("   • F(+∞) = 1")
print("   • F is non-decreasing")
print("   • P(a < X ≤ b) = F(b) - F(a)")

## 6. Joint, Marginal, and Conditional Distributions

In [ ]:
# Visualize joint, marginal, and conditional
from scipy.stats import multivariate_normal

# Create a 2D Gaussian
mean = [0, 0]
cov = [[1, 0.7], [0.7, 1]]  # Correlated

x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))

rv = multivariate_normal(mean, cov)
Z = rv.pdf(pos)

fig = plt.figure(figsize=(15, 5))

# Joint distribution
ax1 = fig.add_subplot(131)
contour = ax1.contourf(X, Y, Z, levels=20, cmap='Blues')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_title('JOINT Distribution f(x, y)\n"The complete picture for two variables"')
ax1.set_aspect('equal')

# Marginal distributions
ax2 = fig.add_subplot(132)
# Marginal of X: integrate out Y
marginal_x = stats.norm.pdf(x, 0, 1)
marginal_y = stats.norm.pdf(y, 0, 1)

ax2.plot(x, marginal_x, 'b-', linewidth=2, label='f_X(x) = ∫f(x,y)dy')
ax2.fill_between(x, marginal_x, alpha=0.3, color='blue')
ax2.plot(y, marginal_y, 'r-', linewidth=2, label='f_Y(y) = ∫f(x,y)dx')
ax2.fill_between(y, marginal_y, alpha=0.3, color='red')
ax2.set_xlabel('Value')
ax2.set_ylabel('Density')
ax2.set_title('MARGINAL Distributions\n"Ignoring one variable"')
ax2.legend()

# Conditional distribution
ax3 = fig.add_subplot(133)
# Conditional Y|X=1
x_given = 1
# For bivariate normal: Y|X=x ~ N(μ_Y + ρ(σ_Y/σ_X)(x - μ_X), σ_Y²(1-ρ²))
rho = 0.7
cond_mean = 0 + rho * (x_given - 0)
cond_var = 1 * (1 - rho**2)

y_vals = np.linspace(-3, 3, 100)
cond_pdf = stats.norm.pdf(y_vals, cond_mean, np.sqrt(cond_var))
uncond_pdf = stats.norm.pdf(y_vals, 0, 1)

ax3.plot(y_vals, uncond_pdf, 'b--', linewidth=2, label='f_Y(y) [marginal]')
ax3.plot(y_vals, cond_pdf, 'r-', linewidth=2, label=f'f(y|X={x_given}) [conditional]')
ax3.fill_between(y_vals, cond_pdf, alpha=0.3, color='red')
ax3.set_xlabel('Y')
ax3.set_ylabel('Density')
ax3.set_title(f'CONDITIONAL Distribution f(y|x={x_given})\n"Given X, what\'s the distribution of Y?"')
ax3.legend()

plt.tight_layout()
plt.show()

print("\n🔑 Key relationships:")
print("   • Marginal: f_X(x) = ∫ f(x,y) dy  ('integrate out' Y)")
print("   • Conditional: f(y|x) = f(x,y) / f_X(x)  (Bayes' rule!)")
print("   • Joint = Conditional × Marginal: f(x,y) = f(y|x) · f_X(x)")

## 7. Sampling: Generating Random Numbers

In [ ]:
# Show that samples approximate the distribution
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

sample_sizes = [10, 100, 1000, 10000, 100000]

x = np.linspace(-4, 4, 1000)
true_pdf = stats.norm.pdf(x, 0, 1)

for i, n in enumerate(sample_sizes):
    ax = axes.flatten()[i]
    
    # Draw samples
    samples = np.random.normal(0, 1, n)
    
    # Plot histogram (normalized to be a density)
    ax.hist(samples, bins=50, density=True, alpha=0.7, color='steelblue', 
            edgecolor='black', label='Samples')
    
    # Overlay true PDF
    ax.plot(x, true_pdf, 'r-', linewidth=2, label='True PDF')
    
    ax.set_xlabel('x')
    ax.set_ylabel('Density')
    ax.set_title(f'n = {n:,} samples')
    ax.set_xlim(-4, 4)
    ax.legend()

# Last plot: show convergence of sample mean
ax = axes.flatten()[5]
cumulative_means = np.cumsum(np.random.normal(0, 1, 10000)) / np.arange(1, 10001)
ax.plot(cumulative_means, 'b-', alpha=0.7)
ax.axhline(y=0, color='r', linestyle='--', linewidth=2, label='True mean = 0')
ax.set_xlabel('Number of samples')
ax.set_ylabel('Sample mean')
ax.set_title('Law of Large Numbers\nSample mean → True mean')
ax.legend()
ax.set_xscale('log')

plt.tight_layout()
plt.show()

print("\n🔑 Key insight: More samples → better approximation of the true distribution")
print("This is the foundation of Monte Carlo methods in ML!")

## Summary

### Key Distinctions

| Concept | Discrete | Continuous |
|---------|----------|------------|
| Distribution function | PMF p(x) | PDF f(x) |
| Meaning | Probability | Density |
| Can exceed 1? | No | Yes |
| P(X = x) | p(x) | 0 |
| P(a ≤ X ≤ b) | Σ p(x) | ∫ f(x) dx |
| Normalization | Σ p(x) = 1 | ∫ f(x) dx = 1 |

### The Transition Recipe

To go from discrete to continuous:
1. Replace Σ with ∫
2. Replace p(x) with f(x)·dx
3. Take the limit as bin width → 0

### What Distributions Give Us

- **Expected values:** E[g(X)] = Σ/∫ g(x)·p/f(x)
- **Probabilities:** P(event) = Σ/∫ over event
- **Sampling:** Generate random values
- **Inference:** Update beliefs with data